## CNN: Fine-Tunning com HDC

### Imports

In [1]:
from modules import utils, globals
import torch
from modules import encoders
from binhd.classifiers import BinHD
from modules.cifake import Cifake
import wisardpkg as wp


In [2]:
# import kagglehub

# # # Download latest version
# path = kagglehub.dataset_download("birdy654/cifake-real-and-ai-generated-synthetic-images")

# print("Path to dataset files:", path)

### Carregando o dataset pré-treinado

In [3]:
model = utils.load_model_from_file("resnet18_cifake_finetuned_float32.pth")
print("Modelo pré-treinado carregado.")
print(model)

/home/marinapiragibe/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/marinapiragibe/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



Modelo 'resnet18_cifake_finetuned_float32.pth' carregado para avaliação de desempenho.
Modelo pré-treinado carregado.
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn

In [4]:
import os
import random
from torch.utils.data import Subset
from torchvision import models, transforms

import torchvision


transform = transforms.Compose([
                                     transforms.Resize(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                          std=[0.229, 0.224, 0.225])
                  ])

train_dataset = torchvision.datasets.ImageFolder(
    root=os.path.join(globals.DATASET_PATH, 'train'),
    transform=transform
)

test_dataset = torchvision.datasets.ImageFolder(
    root=os.path.join(globals.DATASET_PATH, 'test'),
    transform=transform
)

# Cria subsets para testar a lógica do modelo com um número menor do dataset (descomentar para usar)
subset_train_indices = list(range(globals.NUM_SAMPLES_TRAIN_DEBUGGER))
subset_test_indices = list(range(globals.NUM_SAMPLES_TEST_DEBUGGER))

total_train_samples = len(train_dataset)
num_train_to_select = min(globals.NUM_SAMPLES_TRAIN_DEBUGGER, total_train_samples)
subset_train_indices = random.sample(range(total_train_samples), num_train_to_select)

# Pega uma amostra aleatória de índices para o teste
total_test_samples = len(test_dataset)
num_test_to_select = min(globals.NUM_SAMPLES_TEST_DEBUGGER, total_test_samples)
subset_test_indices = random.sample(range(total_test_samples), num_test_to_select)

train_dataset_debugger = Subset(train_dataset, subset_train_indices)
test_dataset_debugger = Subset(test_dataset, subset_test_indices)

train_loader = torch.utils.data.DataLoader(
    train_dataset_debugger,
    batch_size=globals.BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

test_loader = torch.utils.data.DataLoader(
    test_dataset_debugger,
    batch_size=globals.BATCH_SIZE,
    shuffle=False,
    num_workers=0
)
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])



In [5]:
all_features = []
all_labels = []

with torch.no_grad():
    for k, batch in enumerate(train_loader):
        print(f'\r{k+1}/{len(train_loader)}', end='', flush=True) 

        dado, rotulo = batch
        dado = dado.to(globals.DEVICE)
        rotulo = rotulo.to(globals.DEVICE)
        print(rotulo)
        # Extrair features
        features = feature_extractor(dado)

        # Achatar (flatten) o tensor para (batch_size, num_features)
        features = features.view(features.size(0), -1)

        all_features.append(features.cpu())
        all_labels.append(rotulo)

print(all_features)
print(all_labels)

1/13tensor([0, 0, 1, 1, 1, 1, 0, 0])
2/13tensor([1, 0, 0, 0, 0, 1, 1, 1])
3/13tensor([1, 0, 0, 0, 0, 0, 0, 0])
4/13tensor([0, 0, 0, 1, 1, 0, 1, 0])
5/13tensor([1, 0, 0, 1, 0, 1, 0, 1])
6/13tensor([0, 0, 0, 0, 0, 1, 0, 0])
7/13tensor([1, 1, 0, 0, 0, 1, 1, 1])
8/13tensor([1, 0, 1, 1, 1, 1, 1, 0])
9/13tensor([1, 0, 1, 1, 1, 1, 0, 1])
10/13tensor([1, 0, 1, 0, 1, 0, 1, 0])
11/13tensor([0, 0, 0, 0, 1, 1, 0, 1])
12/13tensor([1, 0, 1, 0, 0, 0, 1, 1])
13/13tensor([0, 1, 0, 0])
[tensor([[2.2196, 1.3594, 1.1709,  ..., 0.4722, 1.2890, 0.9199],
        [1.9494, 0.8896, 1.1723,  ..., 1.2122, 1.3804, 1.2615],
        [0.8661, 0.8828, 1.8372,  ..., 0.4204, 1.4998, 1.5154],
        ...,
        [0.2653, 0.1763, 0.5505,  ..., 1.6606, 0.4659, 0.1547],
        [0.7411, 1.3107, 0.1904,  ..., 0.7088, 0.6788, 0.7077],
        [0.5788, 1.7012, 2.2310,  ..., 2.5163, 1.2418, 0.1205]]), tensor([[1.0687, 0.4493, 0.6851,  ..., 0.6031, 0.4936, 0.1024],
        [0.0099, 0.6405, 0.9187,  ..., 1.7196, 0.8884, 0.3000],

In [6]:
import numpy as np

features_array = np.concatenate(all_features, axis=0)
labels_array = np.concatenate(all_labels, axis=0)

print(labels_array)


np.save('features.npy', features_array)
np.save('labels.npy', labels_array)

print("Features salvas em 'features.npy' e labels em 'labels.npy")

[0 0 1 1 1 1 0 0 1 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0
 1 0 1 0 0 0 0 0 1 0 0 1 1 0 0 0 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 1 1 0 1 1 0
 1 0 1 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 0 0 1 1 0 1 0 0]
Features salvas em 'features.npy' e labels em 'labels.npy


In [ ]:
import pandas as pd


cifake = Cifake()
# Verifica as amostras e classes
print(f"Número de samples: {len(cifake.samples)}")
print(f"Classes encontradas: {cifake.features}")

Número de samples: 100
Classes encontradas:       feat_0    feat_1    feat_2    feat_3    feat_4    feat_5    feat_6  \
0   2.219618  1.359371  1.170898  0.466075  0.131603  1.496855  0.984377   
1   1.949351  0.889635  1.172312  0.854865  0.395240  0.961098  1.311359   
2   0.866102  0.882795  1.837225  1.502132  1.152856  0.596776  1.798405   
3   0.999963  2.078024  0.550981  1.369875  3.662720  0.750793  0.352376   
4   0.573586  0.893467  2.847860  1.423239  2.952774  0.403388  1.044446   
..       ...       ...       ...       ...       ...       ...       ...   
95  2.185391  2.359076  0.911664  2.610189  1.880452  0.662483  0.185209   
96  1.222568  1.128433  2.507212  0.214144  1.139926  0.901808  1.023088   
97  0.235074  0.134231  0.329738  0.666386  1.227254  0.456718  1.326144   
98  0.014018  0.062944  1.108675  1.830286  0.178609  0.074424  2.486744   
99  0.352333  0.994777  2.093576  1.099472  0.989623  1.942674  1.075790   

      feat_7    feat_8    feat_9  ...  feat

In [15]:
min_val, max_val = cifake.get_min_max_values()
print(min_val, max_val)

0.0 7.589083194732666


In [22]:
dimension = 1000
num_levels = 500
batch_size = 100
low = 0
high = num_levels
oper = "bind"

In [ ]:
import numpy as np
import pandas as pd
from sklearn.calibration import LabelEncoder

from modules.encoders import RecordEncoder

X = cifake.features

y = cifake.labels
le = LabelEncoder()
y_encoded = torch.tensor(le.fit_transform(y))


0     0
1     0
2     1
3     1
4     1
     ..
95    1
96    0
97    1
98    0
99    0
Name: class, Length: 100, dtype: int64


In [23]:
model = BinHD(dimension, cifake.num_classes)


In [24]:
record_encoder = RecordEncoder(
            out_features=dimension,
            size=X.shape[1], 
            levels=num_levels,
            low=low,
            high=high
        )

In [25]:
y_encoded = torch.tensor(y_encoded).to(globals.DEVICE)

def run_encoders(X, y, device):

    with torch.no_grad():
        print(X.dtypes)

        samples = torch.tensor(X.values).to(device)

        # RecordEncoder
        return record_encoder(samples.detach().clone())



/tmp/ipykernel_38749/3094186489.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_encoded = torch.tensor(y_encoded).to(globals.DEVICE)


In [26]:
from sklearn.model_selection import train_test_split

X_record_encoder = run_encoders(X, y, globals.DEVICE)
labels = torch.tensor(y).to(globals.DEVICE)

X_train, X_test, y_train, y_test = train_test_split(X_record_encoder, labels, test_size=0.3, random_state = 0)

feat_0      float32
feat_1      float32
feat_2      float32
feat_3      float32
feat_4      float32
             ...   
feat_507    float32
feat_508    float32
feat_509    float32
feat_510    float32
feat_511    float32
Length: 512, dtype: object


In [28]:
from sklearn.metrics import accuracy_score

with torch.no_grad():
    model.fit(X_train,y_train)
    predictions = model.predict(X_test.to(torch.int8))  
    acc = accuracy_score(predictions, y_test)
    print("BinHD Record Encoder: Accuracy = ", acc)


BinHD Record Encoder: Accuracy =  0.5333333333333333
